In [1]:

importFrom  <- function(filename, obj_names, where) {
    if (missing(obj_names)) { ## import everything
    source(filename, local = FALSE)
    } else {
        e  <- new.env()
        return_env = new.env()
        source(filename, local = e)
        #if (missing(where)) where = parent.env(parent.env(e))
        for (obj in obj_names) {
            assign(x = obj, value = get(x = obj, envir = e), envir = return_env)
        }
    }
    invisible(TRUE)
    return(return_env)
}

if (! exists("data_4b79235c09cb11eaac9d3af9d395e2bf")) { data_4b79235c09cb11eaac9d3af9d395e2bf = list() }
variable_4b7ac91409cb11ea89ed3af9d395e2bf = c("stage1", "stage2")
data_4b79235c09cb11eaac9d3af9d395e2bf = importFrom("simple/load_data.R", variable_4b7ac91409cb11ea89ed3af9d395e2bf)
for (var_4b7acaae09cb11ea9a833af9d395e2bf in variable_4b7ac91409cb11ea89ed3af9d395e2bf) { assign(var_4b7acaae09cb11ea9a833af9d395e2bf, get(var_4b7acaae09cb11ea9a833af9d395e2bf, env=data_4b79235c09cb11eaac9d3af9d395e2bf))}


In [16]:
#NULL cell where data is loaded

In [17]:
mytest = t.test(stage1[['X1']])$p.value < 0.05

In [4]:
analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf = list();
analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf[["mytest"]] = mytest;


In [19]:
compute_sufficient_statistics <- function(selection) {
    suff_stat = c(sum(stage1[['X1']]), sum(stage1[['X1']]^2))
    names(suff_stat) = c('X1', 'X1_sq')
    return(as.data.frame(t(suff_stat)))
}

compute_estimators <- function(selection) {
    suff_stat = compute_sufficient_statistics(selection)
    value = suff_stat[['X1']] / nrow(stage1)
    names(value) = 'X1'
    indicator = function(sim_selection) {
        return(sim_selection[['mytest']] == TRUE)
    }
    return(list(X1=list(value=value, identifier='X1', indicator=indicator)))
}

estimate_var <- function(selection) {
    variance = var(c(stage1[['X1']]))
    cross = c(variance, 0)
    names(cross) = c('X1', 'X1_sq')
    names(variance) = 'X1'
    return(list(X1=list(identifier='X1', var=variance / nrow(stage1),
                            cross=cross)))
}


resample_data <- function(data, selection) {
    stage1 = data[["stage1"]]
    stage2 = data[["stage2"]]
    
    vars = apply(stage1, 2, var)
    
    means = apply(stage1, 2, mean)
    
    stage1_sim = stage1
    stage1_sim[['X1']] = rnorm(nrow(stage1)) * sqrt(vars[['X1']]) * (0.2 + 2 * runif(1)) + means[['X1']]
    stage2_sim = stage2
    
    sim_env = new.env()
    assign('stage1', as.data.frame(stage1_sim), env=sim_env)
    assign('stage2', as.data.frame(stage2_sim), env=sim_env)
    return(sim_env)
    
}



In [6]:
suff_stat_4c27b54809cb11ea94a63af9d395e2bf = compute_sufficient_statistics(analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf);

In [7]:

library(feather)
filename_4c3b068c09cb11eaaf3f3af9d395e2bf = tempfile()
feather::write_feather(suff_stat_4c27b54809cb11ea94a63af9d395e2bf, filename_4c3b068c09cb11eaaf3f3af9d395e2bf)
file_4c3b05a809cb11eabcc23af9d395e2bf = file(filename_4c3b068c09cb11eaaf3f3af9d395e2bf, "rb")
bin64_4c3b06fa09cb11ea867f3af9d395e2bf = readBin(file_4c3b05a809cb11eabcc23af9d395e2bf, "raw", file.size(filename_4c3b068c09cb11eaaf3f3af9d395e2bf) + 1000)
IRdisplay:::display_raw("application/selective.inference", TRUE, NULL, filename_4c3b068c09cb11eaaf3f3af9d395e2bf, list(encoder="dataframe"))
close(file_4c3b05a809cb11eabcc23af9d395e2bf)
unlink(filename_4c3b068c09cb11eaaf3f3af9d395e2bf)


In [8]:
result_4c61cc5809cb11eaa9503af9d395e2bf = compute_estimators(analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf)
variance_4c61ceb809cb11ea83393af9d395e2bf = estimate_var(analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf)
estimates_4c61ce7409cb11ea8a743af9d395e2bf = c();
names_4c61cdc809cb11eab3a73af9d395e2bf = c();
for (val_4c61cd6409cb11eaa86b3af9d395e2bf in result_4c61cc5809cb11eaa9503af9d395e2bf) {
    names_4c61cdc809cb11eab3a73af9d395e2bf = c(names_4c61cdc809cb11eab3a73af9d395e2bf, val_4c61cd6409cb11eaa86b3af9d395e2bf[['identifier']])
    estimates_4c61ce7409cb11ea8a743af9d395e2bf = c(estimates_4c61ce7409cb11ea8a743af9d395e2bf, val_4c61cd6409cb11eaa86b3af9d395e2bf[['value']])
}
names(estimates_4c61ce7409cb11ea8a743af9d395e2bf) = names_4c61cdc809cb11eab3a73af9d395e2bf
estimates_4c61ce7409cb11ea8a743af9d395e2bf = as.data.frame(as.matrix(t(estimates_4c61ce7409cb11ea8a743af9d395e2bf)))


In [9]:

library(feather)
filename_4c72953609cb11ea898d3af9d395e2bf = tempfile()
feather::write_feather(estimates_4c61ce7409cb11ea8a743af9d395e2bf, filename_4c72953609cb11ea898d3af9d395e2bf)
file_4c72943409cb11eaa0493af9d395e2bf = file(filename_4c72953609cb11ea898d3af9d395e2bf, "rb")
bin64_4c7295a409cb11eab5723af9d395e2bf = readBin(file_4c72943409cb11eaa0493af9d395e2bf, "raw", file.size(filename_4c72953609cb11ea898d3af9d395e2bf) + 1000)
IRdisplay:::display_raw("application/selective.inference", TRUE, NULL, filename_4c72953609cb11ea898d3af9d395e2bf, list(encoder="dataframe"))
close(file_4c72943409cb11eaa0493af9d395e2bf)
unlink(filename_4c72953609cb11ea898d3af9d395e2bf)


In [10]:

library(feather)
filename_4c85c80c09cb11eaab133af9d395e2bf = tempfile()
feather::write_feather(as.data.frame(variance_4c61ceb809cb11ea83393af9d395e2bf[[1]][["var"]]), filename_4c85c80c09cb11eaab133af9d395e2bf)
file_4c85c71e09cb11eabdce3af9d395e2bf = file(filename_4c85c80c09cb11eaab133af9d395e2bf, "rb")
bin64_4c85c87409cb11eaba343af9d395e2bf = readBin(file_4c85c71e09cb11eabdce3af9d395e2bf, "raw", file.size(filename_4c85c80c09cb11eaab133af9d395e2bf) + 1000)
IRdisplay:::display_raw("application/selective.inference", TRUE, NULL, filename_4c85c80c09cb11eaab133af9d395e2bf, list(encoder="dataframe"))
close(file_4c85c71e09cb11eabdce3af9d395e2bf)
unlink(filename_4c85c80c09cb11eaab133af9d395e2bf)


In [11]:

library(feather)
filename_4c9c5d4609cb11ea915a3af9d395e2bf = tempfile()
feather::write_feather(as.data.frame(variance_4c61ceb809cb11ea83393af9d395e2bf[[1]][["cross"]]), filename_4c9c5d4609cb11ea915a3af9d395e2bf)
file_4c9c5c5409cb11eaacf73af9d395e2bf = file(filename_4c9c5d4609cb11ea915a3af9d395e2bf, "rb")
bin64_4c9c5da809cb11eab4193af9d395e2bf = readBin(file_4c9c5c5409cb11eaacf73af9d395e2bf, "raw", file.size(filename_4c9c5d4609cb11ea915a3af9d395e2bf) + 1000)
IRdisplay:::display_raw("application/selective.inference", TRUE, NULL, filename_4c9c5d4609cb11ea915a3af9d395e2bf, list(encoder="dataframe"))
close(file_4c9c5c5409cb11eaacf73af9d395e2bf)
unlink(filename_4c9c5d4609cb11ea915a3af9d395e2bf)


In [13]:
simulated_data_4d37860209cb11eaa5723af9d395e2bf = resample_data(data_4b79235c09cb11eaac9d3af9d395e2bf, analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf)
for(key_4d3787a609cb11ea9b2e3af9d395e2bf in names(simulated_data_4d37860209cb11eaa5723af9d395e2bf)) {
  assign(key_4d3787a609cb11ea9b2e3af9d395e2bf, get(key_4d3787a609cb11ea9b2e3af9d395e2bf, env=simulated_data_4d37860209cb11eaa5723af9d395e2bf))
}

In [14]:

if (! exists("collector_4dcac70a09cb11ea808d3af9d395e2bf")) { collector_4dcac70a09cb11ea808d3af9d395e2bf = list() }

In [15]:
simulated_selection_4dff32ba09cb11eaabb23af9d395e2bf = list();


In [16]:
#NULL cell where data is loaded

In [17]:
mytest = t.test(stage1[['X1']])$p.value < 0.05

In [18]:
simulated_selection_4dff32ba09cb11eaabb23af9d395e2bf[["mytest"]] = mytest;


In [19]:
compute_sufficient_statistics <- function(selection) {
    suff_stat = c(sum(stage1[['X1']]), sum(stage1[['X1']]^2))
    names(suff_stat) = c('X1', 'X1_sq')
    return(as.data.frame(t(suff_stat)))
}

compute_estimators <- function(selection) {
    suff_stat = compute_sufficient_statistics(selection)
    value = suff_stat[['X1']] / nrow(stage1)
    names(value) = 'X1'
    indicator = function(sim_selection) {
        return(sim_selection[['mytest']] == TRUE)
    }
    return(list(X1=list(value=value, identifier='X1', indicator=indicator)))
}

estimate_var <- function(selection) {
    variance = var(c(stage1[['X1']]))
    cross = c(variance, 0)
    names(cross) = c('X1', 'X1_sq')
    names(variance) = 'X1'
    return(list(X1=list(identifier='X1', var=variance / nrow(stage1),
                            cross=cross)))
}


resample_data <- function(data, selection) {
    stage1 = data[["stage1"]]
    stage2 = data[["stage2"]]
    
    vars = apply(stage1, 2, var)
    
    means = apply(stage1, 2, mean)
    
    stage1_sim = stage1
    stage1_sim[['X1']] = rnorm(nrow(stage1)) * sqrt(vars[['X1']]) * (0.2 + 2 * runif(1)) + means[['X1']]
    stage2_sim = stage2
    
    sim_env = new.env()
    assign('stage1', as.data.frame(stage1_sim), env=sim_env)
    assign('stage2', as.data.frame(stage2_sim), env=sim_env)
    return(sim_env)
    
}



In [20]:
suff_stat_4f5a9cda09cb11ea873c3af9d395e2bf = compute_sufficient_statistics(analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf);

In [21]:
collector_4dcac70a09cb11ea808d3af9d395e2bf[['suff_stat_4f5a9cda09cb11ea873c3af9d395e2bf']] = rbind(collector_4dcac70a09cb11ea808d3af9d395e2bf[['suff_stat_4f5a9cda09cb11ea873c3af9d395e2bf']],suff_stat_4f5a9cda09cb11ea873c3af9d395e2bf)


In [22]:
result_4fff93e809cb11ea8eb13af9d395e2bf = compute_estimators(analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf)
indicator_4fff95dc09cb11ea8d4c3af9d395e2bf = c();
names_4fff953a09cb11ea90283af9d395e2bf = c();
for (val_4fff94e209cb11ea96293af9d395e2bf in result_4fff93e809cb11ea8eb13af9d395e2bf) {
    names_4fff953a09cb11ea90283af9d395e2bf = c(names_4fff953a09cb11ea90283af9d395e2bf, val_4fff94e209cb11ea96293af9d395e2bf[['identifier']])
    indicator_4fff95dc09cb11ea8d4c3af9d395e2bf = c(indicator_4fff95dc09cb11ea8d4c3af9d395e2bf, val_4fff94e209cb11ea96293af9d395e2bf[['indicator']](simulated_selection_4dff32ba09cb11eaabb23af9d395e2bf))
}
names(indicator_4fff95dc09cb11ea8d4c3af9d395e2bf) = names_4fff953a09cb11ea90283af9d395e2bf
indicator_4fff95dc09cb11ea8d4c3af9d395e2bf = as.data.frame(as.matrix(t(indicator_4fff95dc09cb11ea8d4c3af9d395e2bf)))


In [23]:
collector_4dcac70a09cb11ea808d3af9d395e2bf[['indicator_4fff95dc09cb11ea8d4c3af9d395e2bf']] = rbind(collector_4dcac70a09cb11ea808d3af9d395e2bf[['indicator_4fff95dc09cb11ea8d4c3af9d395e2bf']],indicator_4fff95dc09cb11ea8d4c3af9d395e2bf)


In [24]:
data_analysis_scooby = function(collector_4dcac70a09cb11ea808d3af9d395e2bf) {
 
#BEGINCELL

    simulated_data_4d37860209cb11eaa5723af9d395e2bf = resample_data(data_4b79235c09cb11eaac9d3af9d395e2bf, analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf)
    for(key_4d3787a609cb11ea9b2e3af9d395e2bf in names(simulated_data_4d37860209cb11eaa5723af9d395e2bf)) {
      assign(key_4d3787a609cb11ea9b2e3af9d395e2bf, get(key_4d3787a609cb11ea9b2e3af9d395e2bf, env=simulated_data_4d37860209cb11eaa5723af9d395e2bf))
    }
#ENDCELL


#BEGINCELL

    
    if (! exists("collector_4dcac70a09cb11ea808d3af9d395e2bf")) { collector_4dcac70a09cb11ea808d3af9d395e2bf = list() }
#ENDCELL


#BEGINCELL

    simulated_selection_4dff32ba09cb11eaabb23af9d395e2bf = list();
    
#ENDCELL


#BEGINCELL

    #NULL cell where data is loaded
#ENDCELL


#BEGINCELL

    mytest = t.test(stage1[['X1']])$p.value < 0.05
#ENDCELL


#BEGINCELL

    simulated_selection_4dff32ba09cb11eaabb23af9d395e2bf[["mytest"]] = mytest;
    
#ENDCELL


#BEGINCELL

    compute_sufficient_statistics <- function(selection) {
        suff_stat = c(sum(stage1[['X1']]), sum(stage1[['X1']]^2))
        names(suff_stat) = c('X1', 'X1_sq')
        return(as.data.frame(t(suff_stat)))
    }
    
    compute_estimators <- function(selection) {
        suff_stat = compute_sufficient_statistics(selection)
        value = suff_stat[['X1']] / nrow(stage1)
        names(value) = 'X1'
        indicator = function(sim_selection) {
            return(sim_selection[['mytest']] == TRUE)
        }
        return(list(X1=list(value=value, identifier='X1', indicator=indicator)))
    }
    
    estimate_var <- function(selection) {
        variance = var(c(stage1[['X1']]))
        cross = c(variance, 0)
        names(cross) = c('X1', 'X1_sq')
        names(variance) = 'X1'
        return(list(X1=list(identifier='X1', var=variance / nrow(stage1),
                                cross=cross)))
    }
    
    
    resample_data <- function(data, selection) {
        stage1 = data[["stage1"]]
        stage2 = data[["stage2"]]
        
        vars = apply(stage1, 2, var)
        
        means = apply(stage1, 2, mean)
        
        stage1_sim = stage1
        stage1_sim[['X1']] = rnorm(nrow(stage1)) * sqrt(vars[['X1']]) * (0.2 + 2 * runif(1)) + means[['X1']]
        stage2_sim = stage2
        
        sim_env = new.env()
        assign('stage1', as.data.frame(stage1_sim), env=sim_env)
        assign('stage2', as.data.frame(stage2_sim), env=sim_env)
        return(sim_env)
        
    }
    
    
#ENDCELL


#BEGINCELL

    suff_stat_4f5a9cda09cb11ea873c3af9d395e2bf = compute_sufficient_statistics(analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf);
#ENDCELL


#BEGINCELL

    collector_4dcac70a09cb11ea808d3af9d395e2bf[['suff_stat_4f5a9cda09cb11ea873c3af9d395e2bf']] = rbind(collector_4dcac70a09cb11ea808d3af9d395e2bf[['suff_stat_4f5a9cda09cb11ea873c3af9d395e2bf']],suff_stat_4f5a9cda09cb11ea873c3af9d395e2bf)
    
#ENDCELL


#BEGINCELL

    result_4fff93e809cb11ea8eb13af9d395e2bf = compute_estimators(analysis_selection_4be5ad4209cb11eaa7103af9d395e2bf)
    indicator_4fff95dc09cb11ea8d4c3af9d395e2bf = c();
    names_4fff953a09cb11ea90283af9d395e2bf = c();
    for (val_4fff94e209cb11ea96293af9d395e2bf in result_4fff93e809cb11ea8eb13af9d395e2bf) {
        names_4fff953a09cb11ea90283af9d395e2bf = c(names_4fff953a09cb11ea90283af9d395e2bf, val_4fff94e209cb11ea96293af9d395e2bf[['identifier']])
        indicator_4fff95dc09cb11ea8d4c3af9d395e2bf = c(indicator_4fff95dc09cb11ea8d4c3af9d395e2bf, val_4fff94e209cb11ea96293af9d395e2bf[['indicator']](simulated_selection_4dff32ba09cb11eaabb23af9d395e2bf))
    }
    names(indicator_4fff95dc09cb11ea8d4c3af9d395e2bf) = names_4fff953a09cb11ea90283af9d395e2bf
    indicator_4fff95dc09cb11ea8d4c3af9d395e2bf = as.data.frame(as.matrix(t(indicator_4fff95dc09cb11ea8d4c3af9d395e2bf)))
    
#ENDCELL


#BEGINCELL

    collector_4dcac70a09cb11ea808d3af9d395e2bf[['indicator_4fff95dc09cb11ea8d4c3af9d395e2bf']] = rbind(collector_4dcac70a09cb11ea808d3af9d395e2bf[['indicator_4fff95dc09cb11ea8d4c3af9d395e2bf']],indicator_4fff95dc09cb11ea8d4c3af9d395e2bf)
    
#ENDCELL

 ; return(collector_4dcac70a09cb11ea808d3af9d395e2bf) }


In [25]:
for(i in 1:8000) { collector_4dcac70a09cb11ea808d3af9d395e2bf = data_analysis_scooby(collector_4dcac70a09cb11ea808d3af9d395e2bf)}